# Over 65 Data

This notebook pulls data from [cso.ie](https://cso.ie) and formats it for our application.

In [56]:
import pandas as pd

CSO data is stored in data cubes. First found the ones I wanted using CSO search, then created function to pull and merge all.

In [57]:
data_cubes = {
    '2011_2016': 'EY015',
    '2022': 'FY010A'
}

dfs = []
for code in data_cubes.values():
    url = f'https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/{code}/CSV/1.0/en'
    df = pd.read_csv(url)
    dfs.append(df)

df = pd.concat(dfs)
df.head()

# Appears different years have different fields. Clean up.
geo_col = next(c for c in df.columns if c in ('Administrative County', 'County and City'))
df = (df.rename(columns={geo_col: 'County'}).query("County != 'State'"))

df.to_csv('base_data.csv', index=False)
df.head()

,C02199V02655,Sex,C03367V04052,County,TLIST(A1),CensusYear,C02076V03371,Age Group,STATISTIC,Statistic Label,UNIT,VALUE,C03789V04537,Administrative Counties
36,-,Both sexes,CW,Carlow,2011,2011,-,All ages,EY015C01,Persons in private households,Number,54187.0,NaN,NaN
37,-,Both sexes,CW,Carlow,2011,2011,-,All ages,EY015C02,Persons living alone,Number,4341.0,NaN,NaN
38,-,Both sexes,CW,Carlow,2011,2011,-,All ages,EY015C03,Persons living alone as a percentage of person...,%,8.0,NaN,NaN
39,-,Both sexes,CW,Carlow,2011,2011,575,65 years and over,EY015C01,Persons in private households,Number,5733.0,NaN,NaN
40,-,Both sexes,CW,Carlow,2011,2011,575,65 years and over,EY015C02,Persons living alone,Number,1506.0,NaN,NaN


### Filter down to just what we want.

- Removing the rows that have percentages instead of number values.
- Select only those living alone.
- Removing rows that have "Ireland" as their area for county.
- Removing "all ages" as we are interested in the 65+

After that, some clean up on field values and column merging.

In [58]:
df = pd.read_csv('base_data.csv')
df = df.query(
    # '`Sex` != "Both sexes" and '
    '`UNIT` == "Number" and '
    '`Statistic Label` == "Persons living alone" and '
    '`Administrative Counties` != "Ireland" and '
    '`Age Group` != "All ages"'
)

df['Administrative Counties'] = df['Administrative Counties'].str.replace('county council', '', case=False).str.strip()
df['County'] = df['County'].str.replace('Limerick City &', 'Limerick City and County', case=False).str.strip()
df['County'] = df['County'].fillna(df['Administrative Counties'])

df = df[['CensusYear', 'County', 'Sex', 'Age Group', 'VALUE']]


df.to_csv('filtered.csv', index=False)

### Extrapolate to 2025 using Compound Annual Growth Rate (CAGR).

This is done using the following algorithm.

$\text{CAGR} = \left( \frac{\text{Final Value}}{\text{Starting Value}} \right)^{\frac{1}{years}} - 1$

Simply: *If growth is smooth and consistent, then what would be the final value?*

I also did a quick check of mortality variations, but it only impacted $\pm1$ so didn't bother to write code to change the values.

In [59]:
df = pd.read_csv('filtered.csv')

base_year = 2016
latest_year = 2022
projected_year = 2025
year_difference = latest_year - base_year

base   = df[df['CensusYear'] == base_year]
latest = df[df['CensusYear'] == latest_year]

merged = base.merge(
    latest,
    on=['County', 'Sex', 'Age Group'],
    suffixes=(f'_{base_year}', f'_{latest_year}')
)

merged['cagr'] = (merged[f'VALUE_{latest_year}'] / merged[f'VALUE_{base_year}']) ** (1 / year_difference) - 1
merged['VALUE'] = merged[f'VALUE_{latest_year}'] * (1 + merged['cagr']) ** 3

projected_df = merged[['County', 'Sex', 'Age Group', 'VALUE']]

projected_df = projected_df.copy()

projected_df['VALUE'] = projected_df['VALUE'].round(0)
projected_df['CensusYear'] = projected_year

projected_df = projected_df[['CensusYear', 'County', 'Sex', 'Age Group', 'VALUE']]

projected_df.to_csv('projected_2025.csv', index=False)

over_65_by_county.csv### Merge in projected to main dataset.

Created two datasets for sanity.

- `over_65_all_by_county.csv` : All values of 65+ onwards.
- `over_65_by_county.csv` : Only the 65+ values. This is the dataset we will use.

In [62]:
filtered = pd.read_csv('filtered.csv')
projected = pd.read_csv('projected_2025.csv')

final = pd.concat([filtered, projected], ignore_index=True)

final = final.sort_values(by=['County', 'CensusYear'])
final['Sex'] = final['Sex'].replace('Both sexes', 'All')

final.to_csv('over_65_all_by_county.csv', index=False)

final = final.query('`Age Group` == "65 years and over"')
final = final[['CensusYear','County','Sex','VALUE']]

final.to_csv('over_65_by_county.csv', index=False)

